In [1]:
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions

from sklearn.preprocessing import StandardScaler
import sklearn

from utils import plot_projections

%matplotlib widget

In [2]:
audio_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/'

wav_files = list(glob.glob(os.path.join(audio_dir, '**/*.mp3')))
wav_files.sort()
print(wav_files)
print(len(wav_files))

['/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_003.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_004.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_005.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_007.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_008.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_009.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_010.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/laser_000_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser

In [3]:
laser_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.mp3')))
laser_files.sort()
print(laser_files)
print(len(laser_files))

['/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_003.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_004.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_005.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_007.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_008.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_009.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_010.mp3']
10


In [4]:
orig_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Original/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*.mp3')))
orig_files.sort()
print(orig_files)
print(len(orig_files))

['/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_003.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_004.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_005.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_006.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_007.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_008.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_009.mp3']
10


In [5]:
print(len(laser_files))
len(orig_files)

10


10

In [6]:
# laser spectograms
nrows = int(len(laser_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,16))

# print(axes)
ax = axes.flatten()
i = 0
for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    stft = librosa.stft(audio_data, n_fft=256)
    S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    
    img = librosa.display.specshow(S_db, x_axis='time', y_axis='log', sr=sr, ax=ax[i])
#     librosa.display.specshow(S_db, x_axis='time', y_axis='log', sr=sr, ax=ax[i])
    i+=1
# plt.title('Mel-frequency spectrogram')
fig.colorbar(img, ax=ax, format='%+2.0f dB')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Original spectograms
nrows = int(len(orig_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,16))

# print(axes)
ax = axes.flatten()
i = 0
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    stft = librosa.stft(audio_data, n_fft=256)
    S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    
    img = librosa.display.specshow(S_db, x_axis='time', y_axis='log', sr=sr, ax=ax[i])
#     librosa.display.specshow(S_db, x_axis='time', y_axis='log', sr=sr, ax=ax[i])
    i+=1
# plt.title('Mel-frequency spectrogram')
fig.colorbar(img, ax=ax, format='%+2.0f dB')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
%%capture
# compute spectograms for both laser and original
laser_spec = []

laser_mu_feat = []
laser_sigma_feat = []
laser_beta_feat = []
for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    stft = librosa.stft(audio_data, n_fft=256)
    S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    
    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for row in S_db:
        
        f = Fitter(row, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]
    
#         print(f_param)
#         print(f.summary())

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    laser_mu_feat.append(mu_feat)
    laser_sigma_feat.append(sigma_feat)
    laser_beta_feat.append(beta_feat)    
    
laser_mu_feat = np.array(laser_mu_feat)
laser_sigma_feat = np.array(laser_sigma_feat)
laser_beta_feat = np.array(laser_beta_feat)

#     laser_spec.append(S_db)

# laser_spec = np.array(laser_spec)
# laser_spec

In [16]:
print(S_db.shape)
print(laser_mu_feat.shape)
print(laser_sigma_feat.shape)
print(laser_beta_feat.shape)

(129, 8607)
(10, 129)
(10, 129)
(10, 129)


In [17]:
%%capture
orig_spec = []

orig_mu_feat = []
orig_sigma_feat = []
orig_beta_feat = []
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    stft = librosa.stft(audio_data, n_fft=256)
    S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    
    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for row in S_db:
        
        f = Fitter(row, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    orig_mu_feat.append(mu_feat)
    orig_sigma_feat.append(sigma_feat)
    orig_beta_feat.append(beta_feat)
    
orig_mu_feat = np.array(orig_mu_feat)
orig_sigma_feat = np.array(orig_sigma_feat)
orig_beta_feat = np.array(orig_beta_feat)

#     orig_spec.append(S_db)

# orig_spec = np.array(orig_spec)
# orig_spec

In [18]:
print(orig_mu_feat.shape)
print(orig_sigma_feat.shape)
print(orig_beta_feat.shape)

(10, 129)
(10, 129)
(10, 129)


In [19]:
def plot_clusters(band_num):
    
    plt.figure(figsize=(10,3))
    
#     mean vs sigma
    plt.subplot(1, 3, 1)
    plt.scatter(orig_mu_feat[:,band_num], orig_sigma_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_mu_feat[:,band_num], laser_sigma_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("Mean")
    plt.ylabel("Variance")
    plt.legend()
    
#     sigma vs beta
    plt.subplot(1, 3, 2)
    plt.scatter(orig_beta_feat[:,band_num], orig_sigma_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_beta_feat[:,band_num], laser_sigma_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("Beta")
    plt.ylabel("Variance")
    plt.legend()
     
#     beta vs mean
    plt.subplot(1, 3, 3)
    plt.scatter(orig_mu_feat[:,band_num], orig_beta_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_mu_feat[:,band_num], laser_beta_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("Mean")
    plt.ylabel("Beta")
    plt.legend()
    
    plt.suptitle("Laser vs Original for {}th Band".format(band_num))
    
    fig = plt.figure(figsize=(10,3))
#     plt.figure(figsize=(20,6))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(orig_mu_feat[:,band_num], orig_sigma_feat[:,band_num], orig_beta_feat[:,band_num], marker='o', label="Original")
    ax.scatter(laser_mu_feat[:,band_num], laser_sigma_feat[:,band_num], laser_beta_feat[:,band_num], marker='x', label="Laser")
    ax.set_xlabel('Mean')
    ax.set_ylabel('Variance')
    ax.set_zlabel('Beta')
    ax.legend()

In [20]:
Band_num = 10 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
Band_num = 20
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
Band_num = 30
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
Band_num = 30
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
Band_num = 40
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
Band_num = 50
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
Band_num = 60
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
Band_num = 70
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
Band_num = 80
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
Band_num = 90
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
Band_num = 100
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
Band_num = 110
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
Band_num = 120
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
Band_num = 125
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#######################  Below Results are for the VAD audios  #######################

In [34]:
laser_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.mp3')))
laser_files.sort()
# print(laser_files)
print(len(laser_files))

72


In [35]:
orig_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Original_cleaned/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*.mp3')))
orig_files.sort()
# print(orig_files)
print(len(orig_files))

74


In [36]:
# laser spectograms
nrows = int(len(laser_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,72))

# print(axes)
ax = axes.flatten()
i = 0
for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    stft = librosa.stft(audio_data, n_fft=256)
    S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    
    img = librosa.display.specshow(S_db, x_axis='time', y_axis='log', sr=sr, ax=ax[i])
#     librosa.display.specshow(S_db, x_axis='time', y_axis='log', sr=sr, ax=ax[i])
    i+=1
# plt.title('Mel-frequency spectrogram')
fig.colorbar(img, ax=ax, format='%+2.0f dB')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
# Original spectograms
nrows = int(len(orig_files)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,72))

# print(axes)
ax = axes.flatten()
i = 0
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    stft = librosa.stft(audio_data, n_fft=256)
    S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    
    img = librosa.display.specshow(S_db, x_axis='time', y_axis='log', sr=sr, ax=ax[i])
#     librosa.display.specshow(S_db, x_axis='time', y_axis='log', sr=sr, ax=ax[i])
    i+=1
# plt.title('Mel-frequency spectrogram')
fig.colorbar(img, ax=ax, format='%+2.0f dB')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
%%capture
# compute spectograms for both laser and original
laser_spec = []

laser_mu_feat = []
laser_sigma_feat = []
laser_beta_feat = []
for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    stft = librosa.stft(audio_data, n_fft=256)
    S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    
    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for row in S_db:
        
        f = Fitter(row, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]
    
#         print(f_param)
#         print(f.summary())

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    laser_mu_feat.append(mu_feat)
    laser_sigma_feat.append(sigma_feat)
    laser_beta_feat.append(beta_feat)    
    
laser_mu_feat = np.array(laser_mu_feat)
laser_sigma_feat = np.array(laser_sigma_feat)
laser_beta_feat = np.array(laser_beta_feat)


In [39]:
print(S_db.shape)
print(laser_mu_feat.shape)
print(laser_sigma_feat.shape)
print(laser_beta_feat.shape)

(129, 414)
(72, 129)
(72, 129)
(72, 129)


In [40]:
%%capture
orig_spec = []

orig_mu_feat = []
orig_sigma_feat = []
orig_beta_feat = []
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    stft = librosa.stft(audio_data, n_fft=256)
    S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    
    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for row in S_db:
        
        f = Fitter(row, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    orig_mu_feat.append(mu_feat)
    orig_sigma_feat.append(sigma_feat)
    orig_beta_feat.append(beta_feat)
    
orig_mu_feat = np.array(orig_mu_feat)
orig_sigma_feat = np.array(orig_sigma_feat)
orig_beta_feat = np.array(orig_beta_feat)


In [41]:
print(orig_mu_feat.shape)
print(orig_sigma_feat.shape)
print(orig_beta_feat.shape)

(74, 129)
(74, 129)
(74, 129)


In [42]:
Band_num = 10 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
Band_num = 20 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
Band_num = 30 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
Band_num = 40 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
Band_num = 50 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
Band_num = 60 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
Band_num = 70 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
Band_num = 80 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
Band_num = 90 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
Band_num = 100 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
Band_num = 110 
plot_clusters(Band_num)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …